In [16]:
import pandas as pd

In [ ]:
#TODO 1: Folter BratData contains empty data
#dla kazdej pary plikow, ktroe maja te sama nazwe ale rozne rozszerzenia
#wczytac zdania z pliku txt i etykiety z pliku ann
#wypierdolic li ijki zaczynajace się od A
#ramka danych id, annotaion_category, if T: begin char, end char, text, if R: arg1, arg2

In [100]:
#wczytaj pliki z folderu BratData
import os


#read txt file
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

path = "../BRAT_Data/Ukraine_after/Articles_for_Ania/2022-06-08_DIPLOMACY_China,_Central_Asia_agree_on_broad_cooperation_cov.txt"
txt = read_txt_file(path)
#print(txt)
print(len(txt))

5625


In [96]:
#read sentences from txt file and save char index od begining and end of sentence
#sencences are in new lines
def read_sentences(txt):
    sentences = []
    start = 0
    end = 0
    for i in range(len(txt)):
        if txt[i] == '\n':
            end = i
            sentences.append((start, end, txt[start:end]))
            start = i+1
    sentences = sentences[7:]
    sentences = pd.DataFrame(sentences, columns=['start', 'end', 'text'])
    return sentences

sentences = read_sentences(txt)
sentences

,start,end,text
0,348,566,China and five Central Asian countries achieve...
1,567,902,Chinese analysts said the outcomes addressed t...
2,903,1117,State Councilor and Foreign Minister Wang Yi a...
3,1118,1299,Wang said that the most important outcome of t...
4,1300,1945,The meeting also adopted four documents includ...
5,1946,2511,Wang said that China will always firmly suppor...
6,2512,2814,Chinese analysts said that the C+C5 foreign mi...
7,2815,2957,The fruitful achievements of the meeting sugge...
8,2958,3398,"Qian Feng, director of the research department..."
9,3399,3486,Cooperation with China has addressed the pragm...


In [87]:
#read ann
ann = read_txt_file(path.replace('.txt', '.ann'))

def clean_ann(ann):
    ids, categories, arg1s, arg2s, texts = [], [], [], [], []
    ann = ann.split('\n')
    while ann[-1] == '':
        ann = ann[:-1]
    ann = [a for a in ann if a[0][0] != 'A']
    for a in ann:
        ann_split = a.split('\t')
        ids.append(ann_split[0])
        args = ann_split[1].split()
        categories.append(args[0])

        if args[0] == 'refers':
            arg1s.append(args[1].split(':')[1])
            arg2s.append(args[2].split(':')[1])
        else:
            arg1s.append(args[1])
            arg2s.append(args[2])
        texts.append(ann_split[2])

    return pd.DataFrame({'id': ids, 'category': categories, 'arg1': arg1s, 'arg2': arg2s, 'text': texts})

ann_df = clean_ann(ann)
ann_df

['T1', 'event 722 746', 'Russia-Ukraine conflict,']
['T2', 'blame 610 721', 'the region\'s most acute challenges of an economic downturn and the threat of a "color revolution" following the']
['R1', 'refers Arg1:T2 Arg2:T1', '']
['T3', 'group_of_countries 346 384', 'China and five Central Asian countries']
['T4', 'praise 385 563', "achieved fruitful cooperation and broad consensus on issues covering the economy, trade and security during Wednesday's third China+Central Asia (C+C5) foreign ministers' meeting"]
['T5', 'praise 587 707', 'the outcomes addressed the region\'s most acute challenges of an economic downturn and the threat of a "color revolution"']
['R2', 'refers Arg1:T4 Arg2:T3', '']
['R3', 'refers Arg1:T5 Arg2:T3', '']
['T6', 'event 3248 3275', 'the Russia-Ukraine conflict']
['T7', 'blame 3102 3249', 'Central Asian countries face great challenges in improving their economies and stabilizing supply chains, which were hit hard by the pandemic and t']
['R4', 'refers Arg1:T7 Arg2

,id,category,arg1,arg2,text
0,T1,event,722,746,"Russia-Ukraine conflict,"
1,T2,blame,610,721,the region's most acute challenges of an econo...
2,R1,refers,T2,T1,
3,T3,group_of_countries,346,384,China and five Central Asian countries
4,T4,praise,385,563,achieved fruitful cooperation and broad consen...
5,T5,praise,587,707,the outcomes addressed the region's most acute...
6,R2,refers,T4,T3,
7,R3,refers,T5,T3,
8,T6,event,3248,3275,the Russia-Ukraine conflict
9,T7,blame,3102,3249,Central Asian countries face great challenges ...


In [88]:
#make realations df
def make_relations_df(ann_df):
    relations = ann_df[ann_df['category'] == 'refers']
    relations = relations.drop(columns=['category', 'text'])
    relations = relations.reset_index(drop=True)
    return relations
relations = make_relations_df(ann_df)
relations

,id,arg1,arg2
0,R1,T2,T1
1,R2,T4,T3
2,R3,T5,T3
3,R4,T7,T6
4,R5,T9,T8
5,R6,T10,T11
6,R7,T13,T12


In [89]:
#make final df
def make_final_df(relations, ann_df, sentences):
    categories2 = []
    arg1s = []
    arg2s = []
    whole_sentences = []

    for _, row in relations.iterrows():
        arg1 = row['arg1']
        arg2 = row['arg2']
        arg1_row = ann_df[ann_df['id'] == arg1]
        arg2_row = ann_df[ann_df['id'] == arg2]

        if not arg1_row['category'].values[0] in ['blame', 'praise']:
            arg1_row, arg2_row = arg2_row, arg1_row

        #check if both arg1 and arg2 are in the same sentence
        arg1_start = int(arg1_row['arg1'])
        arg1_end = int(arg1_row['arg2'])
        arg2_start = int(arg2_row['arg1'])
        arg2_end = int(arg2_row['arg2'])
        arg1_sentence = sentences[(sentences['start'] <= arg1_start) & (sentences['end'] >= arg1_end)]
        arg2_sentence = sentences[(sentences['start'] <= arg2_start) & (sentences['end'] >= arg2_end)]
        if arg1_sentence.empty or arg2_sentence.empty or arg1_sentence.index[0] != arg2_sentence.index[0]:
            continue
        else:
            categories2.append(arg1_row['category'].values[0])
            arg1s.append(arg1_row['text'].values[0])
            arg2s.append(arg2_row['text'].values[0])
            whole_sentences.append(arg1_sentence['text'].values[0])

    return pd.DataFrame({'annotation_category': categories2, 'annotation': arg1s, 'entity': arg2s, 'sentence': whole_sentences})

final_df = make_final_df(relations, ann_df, sentences)
final_df

,annotation_category,annotation,entity,sentence
0,blame,the region's most acute challenges of an econo...,"Russia-Ukraine conflict,",Chinese analysts said the outcomes addressed t...
1,blame,Central Asian countries face great challenges ...,the Russia-Ukraine conflict,"Qian Feng, director of the research department..."
2,blame,some regional countries are worried about poss...,the US,The region has also been under great pressure ...


In [61]:
print(final_df['sentence'][0])
print(final_df['arg1'][0])

Speaking to the Rossiya-1 state television channel on Sunday, Russian President Vladimir Putin warned the US that his country would strike new targets if advanced rocket systems were supplied to Ukraine, adding that the new arms deliveries to Kiev are aimed at "prolonging the conflict.
new arms deliveries to Kiev are aimed at "prolonging the conflict.


In [90]:
#add ner in sentence
import spacy

def add_ner_to_sentence(final_df):
    nlp = spacy.load('en_core_web_sm')
    all_entities = []
    all_entities_annotated = []

    entity_legend = {
        "ORG": "Organizations: companies, agencies, institutions, etc.",
        "PERSON": "People (including fictional)",
        "GPE": "Geopolitical entities: countries, cities, states",
        "LOC": "Non-geopolitical locations: mountain ranges, bodies of water, etc.",
        "NORP": "Nationalities or religious or political groups",
        "EVENT": "Named events: hurricanes, battles, wars, sports events, etc."
    }

    for i in range(len(final_df)):
        sentence = final_df['sentence'][i]
        doc = nlp(sentence)
        entities = [(ent.text) for ent in doc.ents if ent.label_ in entity_legend.keys()]
        entities = list(set(entities))
        entities_annotated = [ ent for ent in entities if ent in final_df['entity'][i]]

        all_entities.append(entities)
        all_entities_annotated.append(entities_annotated)

    final_df['entities_annotated'] = all_entities_annotated

    return final_df

final_df = add_ner_to_sentence(final_df)
final_df

,annotation_category,annotation,entity,sentence,entities_annotated
0,blame,the region's most acute challenges of an econo...,"Russia-Ukraine conflict,",Chinese analysts said the outcomes addressed t...,"[Russia, Ukraine]"
1,blame,Central Asian countries face great challenges ...,the Russia-Ukraine conflict,"Qian Feng, director of the research department...","[Russia, Ukraine]"
2,blame,some regional countries are worried about poss...,the US,The region has also been under great pressure ...,[US]


In [91]:
#atomize entities
def atomize_entities(final_df):
    atomized_df = pd.DataFrame(columns=['annotation_category', 'annotation', 'entity', 'sentence', 'entity_atomized'])
    for i in range(len(final_df)):
        # for every value in entities_annotated make a seperate row and add it to a new df
        if len(final_df['entities_annotated'][i]) > 0:
            for ent in final_df['entities_annotated'][i]:
                new_row = pd.DataFrame([{
                    'annotation_category': final_df['annotation_category'][i],
                    'annotation': final_df['annotation'][i],
                    'entity': final_df['entity'][i],
                    'sentence': final_df['sentence'][i],
                    'entity_atomized': ent
                }])
                atomized_df = pd.concat([atomized_df, new_row], ignore_index=True)
        else:
            new_row = pd.DataFrame([{
                    'annotation_category': final_df['annotation_category'][i],
                    'annotation': final_df['annotation'][i],
                    'entity': final_df['entity'][i],
                    'sentence': final_df['sentence'][i],
                    'entity_atomized': final_df['entity'][i]
                }])
            atomized_df = pd.concat([atomized_df, new_row], ignore_index=True)

    return atomized_df
atomized_df = atomize_entities(final_df)
atomized_df

,annotation_category,annotation,entity,sentence,entity_atomized
0,blame,the region's most acute challenges of an econo...,"Russia-Ukraine conflict,",Chinese analysts said the outcomes addressed t...,Russia
1,blame,the region's most acute challenges of an econo...,"Russia-Ukraine conflict,",Chinese analysts said the outcomes addressed t...,Ukraine
2,blame,Central Asian countries face great challenges ...,the Russia-Ukraine conflict,"Qian Feng, director of the research department...",Russia
3,blame,Central Asian countries face great challenges ...,the Russia-Ukraine conflict,"Qian Feng, director of the research department...",Ukraine
4,blame,some regional countries are worried about poss...,the US,The region has also been under great pressure ...,US


In [93]:
def perform_preprocessing(path):
    txt = read_txt_file(path)
    sentences = read_sentences(txt)
    ann = read_txt_file(path.replace('.txt', '.ann'))
    if len(ann) < 10: return None
    ann_df = clean_ann(ann)
    relations = make_relations_df(ann_df)
    final_df = make_final_df(relations, ann_df, sentences)
    final_df = add_ner_to_sentence(final_df)
    atomized_df = atomize_entities(final_df)
    return atomized_df

In [97]:
path = "../BRAT_Data/Ukraine_after/Articles_for_Ania/2022-06-08_DIPLOMACY_China,_Central_Asia_agree_on_broad_cooperation_cov.txt"

df = perform_preprocessing(path)
df

['T1', 'event 722 746', 'Russia-Ukraine conflict,']
['T2', 'blame 610 721', 'the region\'s most acute challenges of an economic downturn and the threat of a "color revolution" following the']
['R1', 'refers Arg1:T2 Arg2:T1', '']
['T3', 'group_of_countries 346 384', 'China and five Central Asian countries']
['T4', 'praise 385 563', "achieved fruitful cooperation and broad consensus on issues covering the economy, trade and security during Wednesday's third China+Central Asia (C+C5) foreign ministers' meeting"]
['T5', 'praise 587 707', 'the outcomes addressed the region\'s most acute challenges of an economic downturn and the threat of a "color revolution"']
['R2', 'refers Arg1:T4 Arg2:T3', '']
['R3', 'refers Arg1:T5 Arg2:T3', '']
['T6', 'event 3248 3275', 'the Russia-Ukraine conflict']
['T7', 'blame 3102 3249', 'Central Asian countries face great challenges in improving their economies and stabilizing supply chains, which were hit hard by the pandemic and t']
['R4', 'refers Arg1:T7 Arg2

,annotation_category,annotation,entity,sentence,entity_atomized
0,blame,the region's most acute challenges of an econo...,"Russia-Ukraine conflict,",Chinese analysts said the outcomes addressed t...,Russia
1,blame,the region's most acute challenges of an econo...,"Russia-Ukraine conflict,",Chinese analysts said the outcomes addressed t...,Ukraine
2,blame,Central Asian countries face great challenges ...,the Russia-Ukraine conflict,"Qian Feng, director of the research department...",Russia
3,blame,Central Asian countries face great challenges ...,the Russia-Ukraine conflict,"Qian Feng, director of the research department...",Ukraine
4,blame,some regional countries are worried about poss...,the US,The region has also been under great pressure ...,US
